# Decision Tree classification


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.chdir("..")

In [4]:
from IPython.core.debugger import Tracer
import pandas as pd
import numpy as np
import seaborn
import matplotlib.pyplot as plt
from scipy.signal import welch
from IPython.core.debugger import Tracer
from sklearn_pandas import DataFrameMapper, cross_val_score
from p300.feature_extraction import BaseTransformer

In [5]:
df = pd.read_csv("output/instances.csv")

print(df.columns)

Index(['id', 'array_path', 'ch_names', 'event_time', 'event_type', 'index',
       'sfreq', 'subject_id', 'target', 'frequencyextractor__AF3_2.56hz',
       ...
       'subsamplingextractor__AF4_16', 'subsamplingextractor__AF4_17',
       'subsamplingextractor__AF4_18', 'subsamplingextractor__AF4_19',
       'subsamplingextractor__AF4_20', 'subsamplingextractor__AF4_21',
       'subsamplingextractor__AF4_22', 'subsamplingextractor__AF4_23',
       'subsamplingextractor__AF4_24', 'subsamplingextractor__AF4_25'],
      dtype='object', length=471)


Veamos cuántos sujetos tenemos

In [6]:
df.set_index("id", inplace=True)
df.subject_id.unique()

array([29426001, 12521001, 14023001, 18531001, 16893001,   945001,
       23794001,  1414001,  5568001, 27946001, 25922001, 27163001,
       10229001, 16683001, 24540001, 17674001, 23298001, 27030001,
       24888001, 20668001,  1635001, 13431001, 27157001, 24053001,
       11632001,  1609001, 29789001,  8762001, 16943001, 26715001,
          36001,  7615001, 19561001, 29164001, 32459001, 16779001,
       26721001,  2215001, 22109001,  9689001,   499001, 20947001,
       15641001, 31397001,  3703001, 25217001, 11551001, 14998001,
       29063004, 29257001, 20688001,  5510001, 24227001, 23732001,
        9503001, 13252001,  4824001, 16637001,  7330001,   630001,
       29273001, 21601001, 11693001, 16266001, 18112001, 25302001,
       22072001, 27496001, 12702001, 27131001, 12137001,  5857001,
        3109001, 17435001,  6830001, 10882001,  8834001, 27846001,
        7488001, 10924001, 12168001, 13235001,  5224001, 18077001,
       15424001, 23272001, 25871001, 31102001, 17005001, 24101

Tomemos uno: 29164001

In [7]:
non_features = ['id', 'array_path', 'ch_names', 'event_time', 'event_type', 'index',
       'sfreq', 'subject_id', 'target']

def get_data_from_subject(df, subject_id):
    subject_data = df[df.subject_id == subject_id]
    X = subject_data[subject_data.columns.difference(non_features)].as_matrix()
    y = subject_data.target.as_matrix()
    
    return X, y

X, y = get_data_from_subject(df, 29164001)

# DecisionTree

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [12]:
clf = DecisionTreeClassifier(max_depth=100)

In [13]:
cross_val_score(clf, X, y, scoring='roc_auc', cv=10).mean()

0.54266666666666663

# SVM

In [38]:
from sklearn.svm import LinearSVC

clf = LinearSVC?

In [15]:
cross_val_score(clf, X, y, scoring='roc_auc', cv=10).mean()

0.56186666666666663

# Random Forest

In [16]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(100)

In [17]:
cross_val_score(clf, X, y, scoring='roc_auc', cv=10).mean()

0.52585555555555552

In [33]:
clf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

# LDA

In [36]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

clf = LinearDiscriminantAnalysis()

In [37]:
cross_val_score(clf, X, y, scoring='roc_auc', cv=10).mean()

0.55177226813590452

# Balance de las clases

In [27]:
print("Total samples: {} - Total positives {}".format(len(df.target), sum(df.target)))
print("Ratio : {}".format())

Total samples: 287640 - Total positives 47940


Como vemos, están bastante desbalanceadas las clases

# SVM con todos los sujetos

In [28]:
results = []
for subject_id in df.subject_id.unique():
    print("Analyzing {}".format(subject_id))
    X, y = get_data_from_subject(df, subject_id)
    auc = cross_val_score(clf, X, y, scoring='roc_auc').mean()
    precision = cross_val_score(clf, X, y, scoring='precision').mean()
    recall = cross_val_score(clf, X, y, scoring='recall').mean()
    
    results.append({
        'subject_id': subject_id,
        'auc': auc,
        'precision': precision,
        'recall': recall,
    })
    


Analyzing 29426001
Analyzing 12521001
Analyzing 14023001
Analyzing 18531001
Analyzing 16893001
Analyzing 945001
Analyzing 23794001
Analyzing 1414001


/home/jmperez/.pyenv/versions/3.5.1/envs/p300-jmp/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Analyzing 5568001


/home/jmperez/.pyenv/versions/3.5.1/envs/p300-jmp/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Analyzing 27946001
Analyzing 25922001
Analyzing 27163001
Analyzing 10229001
Analyzing 16683001
Analyzing 24540001
Analyzing 17674001
Analyzing 23298001


...

Analyzing 27058001
Analyzing 16003001


/home/jmperez/.pyenv/versions/3.5.1/envs/p300-jmp/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Analyzing 21668001
Analyzing 358001


/home/jmperez/.pyenv/versions/3.5.1/envs/p300-jmp/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jmperez/.pyenv/versions/3.5.1/envs/p300-jmp/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Analyzing 2109001


In [29]:
results_df = pd.DataFrame(results)

In [32]:
results_df.precision.mean()

0.17150196502544177